In [1]:
import stackstac
import numpy as np
import geopandas as gpd
import planetary_computer as pc
from shapely.geometry import mapping, box
from pystac_client import Client as pystac_client

In [2]:
fergana_gdf = gpd.read_file("../data/Train/Fergana_training_samples.shp")
fergana_gdf = fergana_gdf.set_crs(4326)
#fergana_gdf.head(10)

In [3]:
min_x, min_y, max_x, max_y = [round(coord) for coord in fergana_gdf.total_bounds.tolist()]
cell_size = 0.5
x_cells = int((max_x - min_x) / cell_size)
y_cells = int((max_y - min_y) / cell_size)

polygons = []
tile_ids = []
for i in range(x_cells):
    for j in range(y_cells):

        x1 = min_x + i * cell_size
        y1 = min_y + j * cell_size

        poly = box(x1, y1, x1 + cell_size, y1 + cell_size)
        tile_id = f"{x1}_{y1 + cell_size}"
        
        polygons.append(poly)
        tile_ids.append(tile_id)

tiles = gpd.GeoDataFrame({"tile_id": tile_ids, "geometry": polygons}, crs="EPSG:4326")

In [4]:
# Join plots to 0.5° grid cells
joined_gdf = fergana_gdf.sjoin(tiles)
joined_gdf["tile_id"].value_counts()

tile_id
71.0_40.5    42
72.5_40.5    40
72.0_41.5    34
72.5_41.5    34
72.5_41.0    32
72.0_40.5    32
72.0_42.0    32
71.0_42.0    31
71.5_41.0    31
72.0_41.0    30
71.5_41.5    30
71.0_41.0    29
71.0_41.5    28
71.5_40.5    26
72.5_42.0    25
71.5_42.0    24
Name: count, dtype: int64

In [5]:
first_part_df = joined_gdf[joined_gdf["tile_id"] == "71.5_42.0"]
#first_part_df

In [6]:
# Sign into Microsoft planetary computer
stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
sentinel2_client = pystac_client.open(
    stac_url,
    modifier=pc.sign_inplace
)

In [ ]:
bands = ["B2", "B3", "B4", "B5", "B6", "B7", "B8", "B8A", "B11", "B12"]

# Query Sentinel-2 items for Fergana region (2020-2024)
search = sentinel2_client.search(
    collections=["sentinel-2-l2a"],
    bbox=first_part_df.total_bounds.tolist(),
    datetime="2020-01-01/2024-12-31",
    query={"s2:processing_baseline": {"gte": "04.00"}},
    limit=500
)

items = list(search.items())
print(f"Found {len(items)} Sentinel-2 items")

Found 2056 Sentinel-2 items


In [9]:
datacube = stackstac.stack(
    items,
    bounds_latlon=tuple(first_part_df.total_bounds.tolist()),
    assets=bands,
    epsg=4326,
    snap_bounds=False,
    chunksize=(5, len(bands), 1600, 1600)
)
datacube

<xarray.DataArray 'stackstac-3a653d1714a22a4460bab5fd12923dc5' (time: 2056,
                                                                band: 3,
                                                                y: 11004,
                                                                x: 2265)> Size: 1TB
dask.array<fetch_raster_window, shape=(2056, 3, 11004, 2265), dtype=float64, chunksize=(5, 3, 1600, 1600), chunktype=numpy.ndarray>
Coordinates: (12/44)
  * time                                     (time) datetime64[ns] 16kB 2021-...
    id                                       (time) <U54 444kB 'S2A_MSIL2A_20...
  * band                                     (band) <U3 36B 'B8A' 'B11' 'B12'
  * x                                        (x) float64 18kB 71.52 ... 71.99
  * y                                        (y) float64 88kB 41.97 ... 41.52
    s2:reflectance_conversion_factor         (time) float64 16kB 1.029 ... 1.034
    ...                                       ...
    proj:shape                               object 8B {5490}
    gsd                                      float64 8B 20.0
    common_name                              (band) <U7 84B 'rededge' ... 'sw...
    center_wavelength                        (band) float64 24B 0.865 1.61 2.19
    full_width_half_max                      (band) float64 24B 0.033 ... 0.242
    epsg                                     int64 8B 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(71.51588325543031, 41.52335...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 71.52|\n| 0.00,-0.00, 41.97|\n| 0.00, 0.00,...
    resolution_xy:  (0.00020880418943533836, 4.100251366120208e-05)